In [ ]:
from random import shuffle
def sign(x):
    if x < 0:
        return -1
    elif x > 0:
        return 1
    else:
        return 0

class MyGDRegression:
    def __init__(self):
        self.intercept_ = 0.0
        self.coefficients = []
        self.delta = 1

    def fit(self, x, y, learning_rate=0.0001, no_epochs=10000, batch_size=32):
        self.coefficients = [0.0 for _ in range(len(x[0]) + 1)]  # coefficients initialization
        n_samples = len(x)

        for epoch in range(no_epochs):
            for i in range(0, n_samples, batch_size):
                x_batch = x[i:i + batch_size]
                y_batch = y[i:i + batch_size]
                gradients = self.compute_gradients2(x_batch, y_batch)

                for j in range(len(x[0])):
                    self.coefficients[j] -= learning_rate * gradients[j]
                self.coefficients[-1] -= learning_rate * gradients[-1]

        self.intercept_ = self.coefficients[-1]
        self.coefficients = self.coefficients[:-1]

    def eval(self, xi):
        yi = self.coefficients[-1]
        for j in range(len(xi)):
            yi += self.coefficients[j] * xi[j]
        return yi

    def predict(self, x):
        y_computed = [self.eval(xi) for xi in x]
        return y_computed

    def compute_gradients(self, x, y):
        gradients = [0 for _ in range(len(x[0]) + 1)]
        for i in range(len(x)):
            y_computed = self.eval(x[i])
            crt_error = y_computed - y[i]
            for j in range(len(x[0])):
                gradients[j] += crt_error * x[i][j]
            gradients[-1] += crt_error
        for j in range(len(gradients)):
            gradients[j] /= len(x)
        return gradients
    
    def compute_gradients2(self, x, y):
        gradients = [0 for _ in range(len(x[0]) + 1)]

        for i in range(len(x)):
            y_computed = self.eval(x[i])
            crt_error = y_computed - y[i]

            if abs(crt_error) <= self.delta:
                for j in range(len(x[0])):
                    gradients[j] += crt_error * crt_error * x[i][j]
                gradients[-1] += crt_error
            else:
                for j in range(len(x[0])):
                    gradients[j] += (self.delta * sign(crt_error) - 1 / 2 * (self.delta ** 2)) * x[i][j] 
                gradients[-1] += self.delta * sign(crt_error)

        for j in range(len(gradients)):
            gradients[j] /= len(x)

        return gradients

In [ ]:
import os
import csv

from matplotlib import pyplot as plt


def load_data(file_name, input_variable_name, output_variable_name):
    data = []
    data_names = []
    with open(file_name) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                data_names = row
            else:
                data.append(row)
            line_count += 1
    selected_variable = data_names.index(input_variable_name)
    input_list = [float(data[i][selected_variable]) for i in range(len(data))]
    selected_output = data_names.index(output_variable_name)
    output_list = [float(data[i][selected_output]) for i in range(len(data))]

    return input_list, output_list


crtDir = os.getcwd()
filePath = os.path.join(crtDir, 'data', 'world-happiness-report-2017.csv')

inputs, outputs = load_data(filePath, 'Economy..GDP.per.Capita.', 'Happiness.Score')
inputs = [[d] for d in inputs]
print('in:  ', inputs[:5])
print('out: ', outputs[:5])

In [ ]:
def plot_histogram(data, variable_name):
    _ = plt.hist(data, 10)
    plt.title("Histogram of " + variable_name)
    plt.show()


plot_histogram([d[0] for d in inputs], 'Family')
plot_histogram(outputs, 'Happiness Score')

In [ ]:
plt.plot(inputs, outputs, 'ro')
plt.xlabel('Family')
plt.ylabel('happiness')
plt.title('Family vs. happiness')
plt.show()

In [ ]:
import random

indexes = [i for i in range(len(inputs))]
train_sample = random.sample(indexes, k=int(0.75 * len(indexes)))
validation_sample = [i for i in indexes if not i in train_sample]

train_inputs = [inputs[i] for i in train_sample]
train_outputs = [outputs[i] for i in train_sample]

validation_inputs = [inputs[i] for i in validation_sample]
validation_outputs = [outputs[i] for i in validation_sample]

plt.plot(train_inputs, train_outputs, 'ro', label='training data')  #train data are plotted by red and circle sign
plt.plot(validation_inputs, validation_outputs, 'g^',
         label='validation data')  #test data are plotted by green and a triangle sign
plt.title('train and validation data')
plt.xlabel('GDP capita')
plt.ylabel('happiness')
plt.legend()
plt.show()

In [ ]:
regressor = MyGDRegression()

regressor.fit(train_inputs, train_outputs)

predicted = regressor.predict(validation_inputs)
print(predicted)

In [ ]:
no_of_points = 1000
xref = []
t = [d[0] for d in train_inputs]
val = min(t)
step = (max(t) - min(t)) / no_of_points
for i in range(1, no_of_points):
    xref.append(val)
    val += step
yref = [regressor.eval([el]) for el in xref]

plt.plot(train_inputs, train_outputs, 'ro', label='training data')
plt.plot(xref, yref, 'b-', label='learnt model')
plt.title('train data and the learnt model')
plt.xlabel('GDP capita')
plt.ylabel('happiness')
plt.legend()
plt.show()

In [ ]:
computed_validation_outputs = regressor.predict(validation_inputs)

plt.plot(validation_inputs, computed_validation_outputs, 'yo', label='computed test data')
plt.plot(validation_inputs, validation_outputs, 'g^', label='real test data')
plt.title('computed validation and real validation data')
plt.xlabel('GDP capita')
plt.ylabel('happiness')
plt.legend()
plt.show()

In [ ]:
def compute_error():
    error = 0.0
    for t1, t2 in zip(computed_validation_outputs, validation_outputs):
        error += (t1 - t2) ** 2
    error = error / len(validation_outputs)
    print('prediction error (manual): ', error)
    
    error2 = 0.0
    for t1, t2 in zip(computed_validation_outputs, validation_outputs):
        error2 += abs(t1 - t2)
    error2 = error2 / len(validation_outputs)
    print('predicition error2 (manual with MAE)', error2)
    
compute_error()

In [ ]:
n_folds = 3
n_sample = len(inputs)
fold_size = n_sample // n_folds

indexes = [i for i in range(n_sample)]
shuffle(indexes)

for i in range(n_folds):
    validation_indices = indexes[i * fold_size: (i + 1) * fold_size]
    train_indices = [idx for idx in indexes if idx not in validation_indices]
    for j in range(i * fold_size):
        train_indices.append(j)
    for j in range((i + 1) * fold_size, n_sample):
        train_indices.append(j)
    
    train_inputs_v = [inputs[i] for i in train_indices]
    train_outputs_v = [outputs[i] for i in train_indices]

    validation_inputs_v = [inputs[i] for i in validation_indices]
    validation_outputs_v = [outputs[i] for i in validation_indices]
        
    model = MyGDRegression()
    model.fit(train_inputs_v, train_outputs_v)
    prediction = model.predict(validation_inputs_v)
    
    error = 0.0
    for t1, t2 in zip(prediction, validation_outputs_v):
        error += (t1 - t2) ** 2
    error = error / len(validation_outputs_v)
    print('prediction error (manual): ', error)
    
    error2 = 0.0
    for t1, t2 in zip(prediction, validation_outputs_v):
        error2 += abs(t1 - t2)
    error2 = error2 / len(validation_outputs_v)
    print('predicition error2 (manual with MAE)', error2)
